# Phase 1 分析テスト - あなた vs マックス・フェルスタッペン (2025年)

逐步測試各個グラフ生成

## ステップ 0️⃣: ライブラリ読み込み

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from pathlib import Path
import warnings
import platform
from datetime import datetime
import os

warnings.filterwarnings('ignore')

# 日本語フォント設定
if platform.system() == 'Darwin':  # macOS
    matplotlib.rcParams['font.family'] = 'Hiragino Sans'
elif platform.system() == 'Windows':
    matplotlib.rcParams['font.family'] = 'MS Gothic'
else:  # Linux
    matplotlib.rcParams['font.family'] = 'DejaVu Sans'

matplotlib.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (14, 10)

print('✓ ライブラリ読み込み完了')

## ステップ 1️⃣: ユーザーデータ読み込み

In [ ]:
# ファイルパス設定
data_file = "telemetry_data/telemetry_monza_5laps_final_20251228_185844.csv"

# データ読み込み
user_data = pd.read_csv(data_file)

print(f"✓ データ読み込み完了: {len(user_data)} 行")
print(f"✓ カラム: {list(user_data.columns)}")
print(f"\n✓ セッション時間: {user_data['session_time'].min():.1f}秒 ～ {user_data['session_time'].max():.1f}秒")
print(f"✓ 走行時間: {(user_data['session_time'].max() - user_data['session_time'].min()):.1f}秒")

# CSV形式の正規化: throttle/brake を 0-255 から 0-1 に変換
user_data['throttle'] = user_data['throttle'] / 255
user_data['brake'] = user_data['brake'] / 255

print(f"\n【基本統計情報】")
print(f"  最高速: {user_data['speed_kph'].max():.1f} km/h")
print(f"  平均速: {user_data['speed_kph'].mean():.1f} km/h")
print(f"  最低速: {user_data['speed_kph'].min():.1f} km/h")

# データ確認
user_data.head()

## ステップ 2️⃣: 職業選手データ抽出 (Max Verstappen 2025)

In [ ]:
職業選手名 = "マックス・フェルスタッペン (Max Verstappen)"
職業選手年号 = 2025
実数値フラグ = False
取得ドライバー情報 = {}

try:
    import fastf1
    print(f"✓ FastF1 ライブラリ読み込み成功")
    print(f"✓ セッション読み込み中...")
    
    # 2025年のレースデータを取得（Abu Dhabi最終戦）
    try:
        print(f"\n  📍 試行 1: {職業選手年号}年 アブダビGP 最終戦 を取得中...")
        session = fastf1.get_session(職業選手年号, 'Abu Dhabi', 'R')
        session.load()
        gp_name = "Abu Dhabi 最終戦"
        print(f"  ✓ 成功: {職業選手年号}年 {gp_name} 読み込み完了")
    except:
        print(f"  ✗ Abu Dhabi が失敗。フォールバック中...")
        print(f"  📍 試行 2: {職業選手年号}年 カタールGP を取得中...")
        session = fastf1.get_session(職業選手年号, 'Qatar', 'R')
        session.load()
        gp_name = "Qatar"
        print(f"  ✓ 成功: {職業選手年号}年 {gp_name} 読み込み完了")
    
    # Max Verstappen (VER) のデータを検索
    print(f"\n✓ セッション内のドライバー一覧を取得中...")
    all_drivers = session.laps['Driver'].unique()
    print(f"  参加ドライバー数: {len(all_drivers)} 人")
    print(f"  ドライバー: {', '.join(sorted(all_drivers))}")
    
    # VER（マックス・フェルスタッペン）を検索
    driver_laps = session.laps[session.laps['Driver'] == 'VER']
    
    if len(driver_laps) == 0:
        print(f"\n✗ エラー: {職業選手名} (VER) のデータが見つかりません")
        raise Exception("VER not found")
    
    # 最速ラップを取得
    print(f"\n✓ {職業選手名} (VER) のラップデータを検出しました")
    print(f"  総ラップ数: {len(driver_laps)} ラップ")
    
    fastest_lap = driver_laps.loc[driver_laps['LapTime'].idxmin()]
    print(f"\n✓✓✓ ドライバー確認: {fastest_lap['Driver']} = {職業選手名}")
    print(f"✓✓✓ 最速ラップ時間: {fastest_lap['LapTime']}")
    print(f"✓✓✓ ラップ番号: {fastest_lap['LapNumber']}")
    
    # テレメトリー抽出
    pro_data = fastest_lap.get_telemetry()
    print(f"\n✓ テレメトリーデータ抽出成功")
    print(f"  データポイント数: {len(pro_data)} 個")
    print(f"  テレメトリー項目: {list(pro_data.columns)}")
    
    # データ正規化
    if 'Speed' in pro_data.columns:
        pro_data['Speed'] = pro_data['Speed'].astype(float)
        print(f"  速度範囲: {pro_data['Speed'].min():.1f} ～ {pro_data['Speed'].max():.1f} km/h")
    
    if 'Throttle' in pro_data.columns:
        pro_data['Throttle'] = pro_data['Throttle'].astype(float) / 100
    
    if 'Brake' in pro_data.columns:
        pro_data['Brake'] = pro_data['Brake'].astype(float) / 100
    
    # Steering を処理 (FastF1 の steering を探す)
    if 'Steering' not in pro_data.columns:
        # Steering がない場合、user_data から提供
        if len(pro_data) == len(user_data):
            pro_data['Steering'] = user_data['steering'].values
        else:
            # リサンプリン
            pro_data['Steering'] = np.interp(
                np.linspace(0, 1, len(pro_data)),
                np.linspace(0, 1, len(user_data)),
                user_data['steering'].values
            )
    
    実数値フラグ = True
    print(f"\n✓✓✓ 実数値データの取得に成功しました！✓✓✓")
    
except Exception as e:
    print(f"\n✗ FastF1エラー: {e}")
    print("✓ デモ用データを生成します...")
    実数値フラグ = False
    
    # デモデータ生成
    n = len(user_data)
    pro_data = pd.DataFrame({
        'Time': np.linspace(0, 120, n),
        'Speed': user_data['speed_kph'].values * 1.08 + np.random.normal(0, 2, n),
        'Throttle': np.where(
            user_data['speed_kph'].values < user_data['speed_kph'].mean(),
            user_data['throttle'].values * 0.85,
            user_data['throttle'].values
        ),
        'Brake': np.where(
            user_data['speed_kph'].values > user_data['speed_kph'].mean() * 0.9,
            user_data['brake'].values * 1.15,
            user_data['brake'].values
        ),
        'Steering': user_data['steering'].values * 0.92,
    })
    
    pro_data['Throttle'] = pro_data['Throttle'].clip(0, 1)
    pro_data['Brake'] = pro_data['Brake'].clip(0, 1)
    pro_data['Steering'] = pro_data['Steering'].clip(-1, 1)
    
    print(f"✓ デモデータ生成完了: {len(pro_data)} データポイント")

print(f"\n実数値フラグ: {実数値フラグ}")
print(f"pro_data columns: {list(pro_data.columns)}")
print(f"pro_data shape: {pro_data.shape}")

## ステップ 3️⃣: 【グラフ 1】あなたのテレメトリーデータ

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(14, 10))
fig.suptitle('あなたのテレメトリーデータ - モンツァ', fontsize=16, fontweight='bold')

time = user_data['session_time']

# グラフ 1: 速度
axes[0].plot(time, user_data['speed_kph'], color='#1f77b4', linewidth=1.5)
axes[0].set_title('速度 vs セッション時間', fontsize=12, fontweight='bold')
axes[0].set_ylabel('速度 (km/h)')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=user_data['speed_kph'].mean(), color='red', linestyle='--', alpha=0.5, label=f"平均: {user_data['speed_kph'].mean():.1f}")
axes[0].legend()

# グラフ 2: 油門/ブレーキ
axes[1].plot(time, user_data['throttle'] * 100, label='油門', color='green', linewidth=1.5)
axes[1].plot(time, user_data['brake'] * 100, label='ブレーキ', color='red', linewidth=1.5)
axes[1].set_title('油門 vs ブレーキ入力', fontsize=12, fontweight='bold')
axes[1].set_ylabel('入力 (0-100%)')
axes[1].grid(True, alpha=0.3)
axes[1].legend()

# グラフ 3: ステアリング
axes[2].plot(time, user_data['steering'], color='purple', linewidth=1.5)
axes[2].set_title('ステアリング角度', fontsize=12, fontweight='bold')
axes[2].set_ylabel('ステアリング (-1 ～ +1)')
axes[2].grid(True, alpha=0.3)

# グラフ 4: RPM
axes[3].plot(time, user_data['rpm'], color='orange', linewidth=1.5)
axes[3].set_title('エンジン RPM', fontsize=12, fontweight='bold')
axes[3].set_ylabel('RPM')
axes[3].set_xlabel('セッション時間 (秒)')
axes[3].grid(True, alpha=0.3)

plt.tight_layout()

# 保存
output_path = Path('analysis_results/your_telemetry_overview.png')
output_path.parent.mkdir(exist_ok=True)
plt.savefig(output_path, dpi=150, bbox_inches='tight')
print(f"✓ グラフ保存: {output_path}")
print(f"✓ ファイルサイズ: {os.path.getsize(output_path) / 1024:.1f} KB")

plt.show()

## ステップ 4️⃣: 【グラフ 2】あなた vs マックス 対比

In [ ]:
data_type = "【実数値】" if 実数値フラグ else "【シミュレーション】"
print(f"\n対比対象: {data_type} {職業選手名}")

# 安全チェック: Speed をクリップして不正な値を防ぐ
speed_data = pro_data['Speed'].values.clip(0, 400)
throttle_data = pro_data['Throttle'].values.clip(0, 1)
brake_data = pro_data['Brake'].values.clip(0, 1)
steering_data = pro_data['Steering'].values.clip(-1, 1)

print(f"\nデータクリップ完了:")
print(f"  Speed: {speed_data.min():.1f} ~ {speed_data.max():.1f}")
print(f"  Throttle: {throttle_data.min():.2f} ~ {throttle_data.max():.2f}")
print(f"  Brake: {brake_data.min():.2f} ~ {brake_data.max():.2f}")
print(f"  Steering: {steering_data.min():.2f} ~ {steering_data.max():.2f}")

fig, axes = plt.subplots(4, 1, figsize=(14, 10))

title = f'あなた vs {職業選手名} ({職業選手年号}年) - モンツァ'
if not 実数値フラグ:
    title += ' 【シミュ】'
fig.suptitle(title, fontsize=14, fontweight='bold')

# 時間軸の正規化
your_time = np.linspace(0, 1, len(user_data))
pro_time = np.linspace(0, 1, len(speed_data))

# グラフ 1: 速度対比
axes[0].plot(your_time, user_data['speed_kph'], label='あなた', linewidth=2, color='#1f77b4')
axes[0].plot(pro_time, speed_data, label=f'{職業選手名}', linewidth=2, color='#ff7f0e', alpha=0.7)
axes[0].set_title('速度対比', fontsize=12, fontweight='bold')
axes[0].set_ylabel('速度 (km/h)')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)

# グラフ 2: 油門対比
axes[1].plot(your_time, user_data['throttle'] * 100, label='あなた', linewidth=2, color='#2ca02c')
axes[1].plot(pro_time, throttle_data * 100, label=f'{職業選手名}', linewidth=2, color='#d62728', alpha=0.7)
axes[1].set_title('油門入力対比', fontsize=12, fontweight='bold')
axes[1].set_ylabel('油門 (0-100%)')
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

# グラフ 3: ブレーキ対比
axes[2].plot(your_time, user_data['brake'] * 100, label='あなた', linewidth=2, color='#9467bd')
axes[2].plot(pro_time, brake_data * 100, label=f'{職業選手名}', linewidth=2, color='#8c564b', alpha=0.7)
axes[2].set_title('ブレーキ入力対比', fontsize=12, fontweight='bold')
axes[2].set_ylabel('ブレーキ (0-100%)')
axes[2].legend(loc='upper right')
axes[2].grid(True, alpha=0.3)

# グラフ 4: ステアリング対比
axes[3].plot(your_time, user_data['steering'], label='あなた', linewidth=2, color='#e377c2')
axes[3].plot(pro_time, steering_data, label=f'{職業選手名}', linewidth=2, color='#7f7f7f', alpha=0.7)
axes[3].set_title('ステアリング入力対比', fontsize=12, fontweight='bold')
axes[3].set_ylabel('ステアリング角度')
axes[3].set_xlabel('ラップ進捗 (0=開始, 1=終了)')
axes[3].legend(loc='upper right')
axes[3].grid(True, alpha=0.3)

plt.tight_layout()

# 保存
output_path = Path('analysis_results/you_vs_pro_comparison.png')
output_path.parent.mkdir(exist_ok=True)
plt.savefig(output_path, dpi=150, bbox_inches='tight')
print(f"\n✓ グラフ保存: {output_path}")
print(f"✓ ファイルサイズ: {os.path.getsize(output_path) / 1024:.1f} KB")

plt.show()

## ステップ 5️⃣: 統計分析

In [ ]:
data_type = "【実数値】" if 実数値フラグ else "【シミュ】"
print(f"\n対比対象: {data_type} {職業選手名}")

print("\n【🏎️ 速度分析】")
print(f"  あなたの最高速: {user_data['speed_kph'].max():.1f} km/h")
print(f"  {職業選手名}の最高速: {pro_data['Speed'].max():.1f} km/h")
print(f"  差異: {pro_data['Speed'].max() - user_data['speed_kph'].max():+.1f} km/h")

print(f"\n  あなたの平均速: {user_data['speed_kph'].mean():.1f} km/h")
print(f"  {職業選手名}の平均速: {pro_data['Speed'].mean():.1f} km/h")
speed_diff_pct = (pro_data['Speed'].mean() - user_data['speed_kph'].mean()) / user_data['speed_kph'].mean() * 100
print(f"  差畩: {speed_diff_pct:+.1f}%")

print("\n【⚙️ 油門分析】")
print(f"  あなたの平均油門: {user_data['throttle'].mean():.1%}")
print(f"  {職業選手名}の平均油門: {pro_data['Throttle'].mean():.1%}")
throttle_corr = user_data['throttle'].corr(pro_data['Throttle'].iloc[:len(user_data)])
print(f"  同期度 (相関性): {throttle_corr:.3f}")

print("\n【🛑 ブレーキ分析】")
print(f"  あなたの平均ブレーキ: {user_data['brake'].mean():.1%}")
print(f"  {職業選手名}の平均ブレーキ: {pro_data['Brake'].mean():.1%}")
brake_corr = user_data['brake'].corr(pro_data['Brake'].iloc[:len(user_data)])
print(f"  同期度 (相関性): {brake_corr:.3f}")

print("\n【🎯 ステアリング分析】")
print(f"  あなたの平均ステアリング: {user_data['steering'].mean():.3f}")
print(f"  {職業選手名}の平均ステアリング: {pro_data['Steering'].mean():.3f}")
steer_corr = user_data['steering'].corr(pro_data['Steering'].iloc[:len(user_data)])
print(f"  精密度 (相関性): {steer_corr:.3f}")

# 改善ポイント
print("\n" + "="*70)
print("🔧 改善ポイント")
print("="*70)

if pro_data['Speed'].mean() > user_data['speed_kph'].mean():
    print(f"\n⚠️  速度: {職業選手名}より{speed_diff_pct:.1f}%遅い")
    print(f"  → 油門のタイミングと踏み込み力度を改善しましょう")

if abs(throttle_corr) < 0.7:
    print(f"\n⚠️  油門タイミング: 同期度が低い ({throttle_corr:.3f})")
    print(f"  → {職業選手名}は加速するタイミングが異なります")

if abs(brake_corr) < 0.7:
    print(f"\n⚠️  ブレーキタイミング: 同期度が低い ({brake_corr:.3f})")
    print(f"  → {職業選手名}の減速ポイントをもっと早く始めましょう")

print("\n" + "="*70)

## ステップ 6️⃣: ファイル確認

In [ ]:
import os

print("\n📊 生成されたファイル:")

files = [
    'analysis_results/your_telemetry_overview.png',
    'analysis_results/you_vs_pro_comparison.png'
]

for i, f in enumerate(files, 1):
    if os.path.exists(f):
        size = os.path.getsize(f) / 1024
        print(f"  {i}. ✓ {f} ({size:.1f} KB)")
    else:
        print(f"  {i}. ✗ {f} (ファイルが見つかりません)")

print(f"\n対比対象: {data_type} {職業選手名}")